<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/regression/Energy_price_demand_and_potential_supply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets

* Wind and solar power capacity Poland <br>
https://www.statista.com/statistics/1003492/poland-wind-and-solar-power-capacity-forecast/

* Predict solar energy <br>
* https://towardsdatascience.com/predicting-solar-power-output-using-machine-learning-techniques-56e7959acb1f <br>
* https://www.omnicalculator.com/ecology/solar-panel#:~:text=To%20find%20the%20solar%20panel,day%20is%20just%20an%20average.

* Wind electric power atlas <br>
https://globalwindatlas.info/en/area/Poland/Ma%C5%82opolskie